In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 894 rows, 894 columns and 894 nonzeros
Model has 400065 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e+01, 4e+07]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 894 rows and 0 columns
Presolve time: 0.02s
Presolved: 0 rows, 894 columns, 0 nonzeros
Presolved model has 400065 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 764
 AA' NZ     : 2.915e+05
 Factor NZ  : 2.922e+05 (roughly 3 MBytes of memory)
 Factor Ops : 1.489e+08 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.19455382e+10 -2.13211538e+05  5.92e+04 1.00e+04  2.16e+06     0s
   1  -3.27086927e+09 -6.09775799e+07  3.84e+03 6.49e+02  3.39e+05     0s
   2  -4.91480804e+07 -2.78905882e+07  6.68e+01 1.13e

In [2]:
# x = GLS_Apr_weekday_PM.x; # sample matrix, each column is a link flow vector sample; number_of_links * K
# S = GLS_Apr_weekday_PM.S; # sample covariance matrix
# A = GLS_Apr_weekday_PM.A; # link_route incidence matrix
P = GLS_Apr_weekday_PM.P; # route_choice_probability_matrix
# Q = GLS_Apr_weekday_PM.Q; 
L = GLS_Apr_weekday_PM.L; # dimension of xi
number_of_routes = GLS_Apr_weekday_PM.number_of_routes;
number_of_links = GLS_Apr_weekday_PM.number_of_links;

In [3]:
# A = sparse(A);
P = sparse(P);
# Q = sparse(Q);

In [4]:
# K = size(x, 2)

# inv_S = inv(S)

# A_t = transpose(A)

# b = sum([A_t * inv_S * x[:, k] for k = 1:K])

In [5]:
using JuMP

### implement GLS method to estimate OD demand matrix

In [6]:
# model = Model(solver=IpoptSolver())

# @variable(model, xi[1:L] >= 0)

# # Set objective: (K/2) xi' * Q * xi - b' * xi
# obj = 0
# for i = 1:L
#     for j = 1:L
#         obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
#     end
# end
# for l = 1:L
#     obj += - b[l] * xi[l]
# end

# @NLobjective(model, Min, obj) 

# solve(model)

In [7]:
# xi_list = getvalue(xi)

xi_list = GLS_Apr_weekday_PM.xi_list

894-element Array{Any,1}:
   0.0106748  
   0.00181372 
   0.00538471 
  15.5571     
   0.0026123  
   0.0274217  
   0.00258339 
   0.00202091 
   0.000828581
 487.139      
 287.629      
   0.00235297 
   0.00190731 
   ⋮          
   8.03428e-6 
   8.8706e-6  
   8.57096e-6 
   8.85979e-6 
   2.31588e-5 
   1.58445e-5 
   1.15725e-5 
   6.88696e-7 
   1.14576e-5 
   3.52878e-5 
   7.88324e-7 
   5.7259e-7  

In [ ]:
mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)

In [ ]:
getvalue(lam)

In [ ]:
getobjectivevalue(mGLSJulia)